In [2]:
import pandas as pd
import numpy as np
from googlemaps import Client as GoogleMaps

# step 1 - generate a distance matrix


I wanted to visualize the distance between each city (any kind of settelment) in israel to its nearesrt hospital.
to that end, I wanted to create a distance matrix table in whice the indexies are israel's cities and the column heads are 
hospitals. 
this would allow me to easly find the closest hospital to each city (minimum of each raw)

In the previus steps I've generated two tables, now storedin *.csv files:
1. lamas_2018_WGS.csv - A LAMAS report with all cities of israel - and WGS coords I've addded
2. hospitals.csv - All hospitals in israel and their WGS coords I've generated using GMAPS API

In [3]:
#Loading hospitals file. note that the 'code' column is an un-official code I assigned, but I will use it from here on to 
#identify each hospital
file = r'..\csv\hospitals.csv'
hospitals = pd.read_csv(file)
hospitals.head()

,code,hebName,engName,lat,long
0,0,שיבא,Shiba,32.046710,34.842852
1,1,רמבם,Rambam,32.832978,34.985706
2,2,אסף הרופא,Asaf Harofe,31.966053,34.839418
3,3,וולפסון,Wolfson,32.036031,34.762707
4,4,זיו,Ziv,32.954157,35.492657


In [4]:
#Loading the lamas file with all cities in israel and WGS coords
file2 = r'..\csv\lamas_2018_with_wgs.csv'
lamas = pd.read_csv(file2)

In [5]:
lamas.head()

,code,nameHeb,pronaunce,district,shire,naturalArea,monicipal,metro,religion,population,...,altitude,orgazinationRelated,plansComit,policeArea,year,nameEng,localCuncil,name,lat,long
0,967,אבו ג'ווייעד (שבט),ABU JUWEI'ID,6,62,623.0,NaN,NaN,3.0,NaN,...,2040057100,NaN,699,15003711,2018,Abu Juway'ad,NaN,Abu Juway'ad,_,_
1,472,אבו גוש,ABU GHOSH,1,11,111.0,99.0,444.0,2.0,7543.0,...,2105263481,598.0,152,10002475,2018,Abu Ghosh,NaN,Abu Ghosh,31.806293,35.109193
2,473,אבו סנאן,ABU SINAN,2,24,245.0,99.0,NaN,2.0,13915.0,...,2160776284,19.0,252,10004315,2018,Abu Sinan,NaN,Abu Sinan,32.95803,35.171969
3,935,אבו סריחאן (שבט),ABU SUREIHAN,6,62,623.0,NaN,NaN,3.0,NaN,...,1865057450,NaN,699,10001937,2018,Abu Surayhan,NaN,Abu Surayhan,_,_
4,958,אבו עבדון (שבט),ABU ABDUN,6,62,623.0,NaN,NaN,3.0,NaN,...,1850057900,NaN,699,10001937,2018,Abu 'Abdun,NaN,Abu 'Abdun,31.301397,34.841655


In [6]:
#in the next steps I created a distance matrix in which each index is a city code, and each column head is a hospital code
disMat = pd.DataFrame({'code': lamas['code'], 'name' : lamas['name']})
disMat.index= lamas['code']

dis_to_hospital = pd.DataFrame(np.zeros((len(disMat), len(hospitals))))
dis_to_hospital.columns = hospitals['code']
dis_to_hospital.index = disMat.index

disMat = pd.concat([disMat,dis_to_hospital], join ='outer', copy = False, axis = 1)
disMat.head()

,code,name,0,1,2,3,4,5,6,7,...,19,20,21,22,23,24,25,26,27,28
code,,,,,,,,,,,,,,,,,,,,,
967,967,Abu Juway'ad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
472,472,Abu Ghosh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473,473,Abu Sinan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
935,935,Abu Surayhan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
958,958,Abu 'Abdun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# step 2 - calculate distance from each city to each hospital 

So now that we have the empty distance matrix, we can iterate over all cities in the list. For each city, we will itterate over all hospitals, and fill the distance matrix using the GMAPS Distance API.

For that you need to generate a GMAPS Distance API Key from your google cloud. this process cost money per each query, but google gives you free trials. (you still need to a a billing method)
Again, just google "get GMAPS Distance API credentiale

In [9]:
hospitals_coords = hospitals[['code', 'lat', 'long']]

In [14]:
#The getDist function itterate over cities and hospitals WGS coords to get thier driving distance.
#Note that after each city the function backup the current distance matrix to a csv file after each city itteration
def getDist(israel, hospitals_coords, disMat):
    '''
    INPUT :
        israel - df of israel with lat/long, indexes by lamas codes
        hospital_coord df with hospitals codes lat/long  position
        disMat - a distance matrix with cities as raws index and hospitals codes as columns
            it should be filled with GMAPS distance from each city to each jospital
    OUTPUT - distance matrix (disMat)
    '''
    gmaps = GoogleMaps('Enter Your Key Here')
    i = 0
    n_of_cities = len(israel)
    
    #itterate cities and hospitals
    for city_code in israel.index:
        city_lat = israel.loc[city_code,:]['lat']
        city_long = israel.loc[city_code,:]['long']
        origins = (city_lat,city_long)
        
        for hospital_code in hospitals_coords.index:
            hospital_lat = hospitals_coords.loc[hospital_code,:]['lat']
            hospital_long = hospitals_coords.loc[hospital_code,:]['long']
            destination = (hospital_lat, hospital_long)
            try:
                result = gmaps.distance_matrix(origins, destination, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
                disMat.loc[city_code, hospital_code] = result
                
                     
            except:
                pass
        i+=1
        
        #After each city is over - BACKUP
        print(f'finished {i} of {n_of_cities} cities')
        path = r'\3 Israel Hospitals\backup\\'
        name = str(city_code) + '.csv'
        new_path = path+name
        disMat.to_csv(path+name)
        
    return (israel, hospitals_coords, disMat)
                                                                                     

In [15]:
results = getDist(israel, hospitals_coords, disMat)


finished 1 of 1220 cities
finished 2 of 1220 cities
finished 3 of 1220 cities
finished 4 of 1220 cities
finished 5 of 1220 cities
finished 6 of 1220 cities
finished 7 of 1220 cities
finished 8 of 1220 cities
finished 9 of 1220 cities
finished 10 of 1220 cities
finished 11 of 1220 cities
finished 12 of 1220 cities
finished 13 of 1220 cities
finished 14 of 1220 cities
finished 15 of 1220 cities
finished 16 of 1220 cities
finished 17 of 1220 cities
finished 18 of 1220 cities
finished 19 of 1220 cities
finished 20 of 1220 cities
finished 21 of 1220 cities
finished 22 of 1220 cities
finished 23 of 1220 cities
finished 24 of 1220 cities
finished 25 of 1220 cities
finished 26 of 1220 cities
finished 27 of 1220 cities
finished 28 of 1220 cities
finished 29 of 1220 cities
finished 30 of 1220 cities
finished 31 of 1220 cities
finished 32 of 1220 cities
finished 33 of 1220 cities
finished 34 of 1220 cities
finished 35 of 1220 cities
finished 36 of 1220 cities
finished 37 of 1220 cities
finished 3

finished 298 of 1220 cities
finished 299 of 1220 cities
finished 300 of 1220 cities
finished 301 of 1220 cities
finished 302 of 1220 cities
finished 303 of 1220 cities
finished 304 of 1220 cities
finished 305 of 1220 cities
finished 306 of 1220 cities
finished 307 of 1220 cities
finished 308 of 1220 cities
finished 309 of 1220 cities
finished 310 of 1220 cities
finished 311 of 1220 cities
finished 312 of 1220 cities
finished 313 of 1220 cities
finished 314 of 1220 cities
finished 315 of 1220 cities
finished 316 of 1220 cities
finished 317 of 1220 cities
finished 318 of 1220 cities
finished 319 of 1220 cities
finished 320 of 1220 cities
finished 321 of 1220 cities
finished 322 of 1220 cities
finished 323 of 1220 cities
finished 324 of 1220 cities
finished 325 of 1220 cities
finished 326 of 1220 cities
finished 327 of 1220 cities
finished 328 of 1220 cities
finished 329 of 1220 cities
finished 330 of 1220 cities
finished 331 of 1220 cities
finished 332 of 1220 cities
finished 333 of 1220

finished 591 of 1220 cities
finished 592 of 1220 cities
finished 593 of 1220 cities
finished 594 of 1220 cities
finished 595 of 1220 cities
finished 596 of 1220 cities
finished 597 of 1220 cities
finished 598 of 1220 cities
finished 599 of 1220 cities
finished 600 of 1220 cities
finished 601 of 1220 cities
finished 602 of 1220 cities
finished 603 of 1220 cities
finished 604 of 1220 cities
finished 605 of 1220 cities
finished 606 of 1220 cities
finished 607 of 1220 cities
finished 608 of 1220 cities
finished 609 of 1220 cities
finished 610 of 1220 cities
finished 611 of 1220 cities
finished 612 of 1220 cities
finished 613 of 1220 cities
finished 614 of 1220 cities
finished 615 of 1220 cities
finished 616 of 1220 cities
finished 617 of 1220 cities
finished 618 of 1220 cities
finished 619 of 1220 cities
finished 620 of 1220 cities
finished 621 of 1220 cities
finished 622 of 1220 cities
finished 623 of 1220 cities
finished 624 of 1220 cities
finished 625 of 1220 cities
finished 626 of 1220

finished 884 of 1220 cities
finished 885 of 1220 cities
finished 886 of 1220 cities
finished 887 of 1220 cities
finished 888 of 1220 cities
finished 889 of 1220 cities
finished 890 of 1220 cities
finished 891 of 1220 cities
finished 892 of 1220 cities
finished 893 of 1220 cities
finished 894 of 1220 cities
finished 895 of 1220 cities
finished 896 of 1220 cities
finished 897 of 1220 cities
finished 898 of 1220 cities
finished 899 of 1220 cities
finished 900 of 1220 cities
finished 901 of 1220 cities
finished 902 of 1220 cities
finished 903 of 1220 cities
finished 904 of 1220 cities
finished 905 of 1220 cities
finished 906 of 1220 cities
finished 907 of 1220 cities
finished 908 of 1220 cities
finished 909 of 1220 cities
finished 910 of 1220 cities
finished 911 of 1220 cities
finished 912 of 1220 cities
finished 913 of 1220 cities
finished 914 of 1220 cities
finished 915 of 1220 cities
finished 916 of 1220 cities
finished 917 of 1220 cities
finished 918 of 1220 cities
finished 919 of 1220

finished 1171 of 1220 cities
finished 1172 of 1220 cities
finished 1173 of 1220 cities
finished 1174 of 1220 cities
finished 1175 of 1220 cities
finished 1176 of 1220 cities
finished 1177 of 1220 cities
finished 1178 of 1220 cities
finished 1179 of 1220 cities
finished 1180 of 1220 cities
finished 1181 of 1220 cities
finished 1182 of 1220 cities
finished 1183 of 1220 cities
finished 1184 of 1220 cities
finished 1185 of 1220 cities
finished 1186 of 1220 cities
finished 1187 of 1220 cities
finished 1188 of 1220 cities
finished 1189 of 1220 cities
finished 1190 of 1220 cities
finished 1191 of 1220 cities
finished 1192 of 1220 cities
finished 1193 of 1220 cities
finished 1194 of 1220 cities
finished 1195 of 1220 cities
finished 1196 of 1220 cities
finished 1197 of 1220 cities
finished 1198 of 1220 cities
finished 1199 of 1220 cities
finished 1200 of 1220 cities
finished 1201 of 1220 cities
finished 1202 of 1220 cities
finished 1203 of 1220 cities
finished 1204 of 1220 cities
finished 1205 

In [23]:
# results[0].to_csv('israel.csv')
# results[1].to_csv('hospitals_codes')
results[2].to_csv(r'..\csv\isrHospitalDisMat.csv')